In [ ]:
!pip install -U tensorflow -q
!pip install -U unsloth vllm -q
!pip install bitsandbytes accelerate peft -q

In [2]:
import unsloth
from unsloth import FastModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only
import argparse
import logging
import sys
from transformers import TrainingArguments, DataCollatorForSeq2Seq
import os, glob, shutil, logging
import torch
from datasets import load_dataset
from huggingface_hub import login
from trl import SFTTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


INFO 10-27 20:23:08 [__init__.py:216] Automatically detected platform cuda.
ERROR 10-27 20:23:09 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
OUTPUT_DIR = "gemma-3-finetuned"
MODEL_NAME = "unsloth/gemma-3-4b-it"

# Info about the system

In [4]:
# Log system info
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
GPU memory: 15.8 GB


In [5]:
model, tokenizer = FastModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = 2048,
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # A bit more accurate, uses 2x memory
    full_finetuning = False # Whether to fine-tune all model weights or just adapters (if available)
)

==((====))==  Unsloth 2025.10.10: Fast Gemma3 patching. Transformers: 4.56.2. vLLM: 0.11.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [6]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Should leave on!
    finetune_mlp_modules       = True,  # Should leave on!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
# Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [rewoo/planner_instruction_tuning_2k](https://huggingface.co/datasets/rewoo/planner_instruction_tuning_2k) dataset composed of <**Instruction, Input, Output**>.

Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use `get_chat_template` function to get the correct chat template. Unsloth natively supports `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [12]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [8]:
from datasets import load_dataset
dataset = load_dataset("rewoo/planner_instruction_tuning_2k", split = "train")

# To reduce the training time, we will use a smaller dataset. You can remove this line to use the full dataset.
dataset = dataset.select(range(100))

dataset = dataset.train_test_split(test_size=0.1, seed=3407)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

README.md:   0%|          | 0.00/175 [00:00<?, ?B/s]

mix_insft_2k.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2044 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-3-4b-it", use_fast=True)
tokenizer.get_chat_template()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

'{{ bos_token }}\n{%- if messages[0][\'role\'] == \'system\' -%}\n    {%- if messages[0][\'content\'] is string -%}\n        {%- set first_user_prefix = messages[0][\'content\'] + \'\n\n\' -%}\n    {%- else -%}\n        {%- set first_user_prefix = messages[0][\'content\'][0][\'text\'] + \'\n\n\' -%}\n    {%- endif -%}\n    {%- set loop_messages = messages[1:] -%}\n{%- else -%}\n    {%- set first_user_prefix = "" -%}\n    {%- set loop_messages = messages -%}\n{%- endif -%}\n{%- for message in loop_messages -%}\n    {%- if (message[\'role\'] == \'user\') != (loop.index0 % 2 == 0) -%}\n        {{ raise_exception("Conversation roles must alternate user/assistant/user/assistant/...") }}\n    {%- endif -%}\n    {%- if (message[\'role\'] == \'assistant\') -%}\n        {%- set role = "model" -%}\n    {%- else -%}\n        {%- set role = message[\'role\'] -%}\n    {%- endif -%}\n    {{ \'<start_of_turn>\' + role + \'\n\' + (first_user_prefix if loop.first else "") }}\n    {%- if message[\'conte

In [10]:
def formatting_prompts_func(examples):
    """Converte il dataset in formato conversazionale Gemma-3"""
    texts = []

    for instr, inp, out in zip(examples["instruction"], examples["input"], examples["output"]):
        # Costruisci il prompt utente
        if inp.strip():
            user_content = f"{instr}\n\nInput: {inp}"
        else:
            user_content = instr

        # Formato conversazionale
        conversation = [
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": out}
        ]

        text = tokenizer.apply_chat_template(
            conversation,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)

    return {"text": texts}

train_dataset = train_dataset.map(formatting_prompts_func, batched=True, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(formatting_prompts_func, batched=True, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [11]:
print(train_dataset[0]['text'])

<bos><start_of_turn>user
For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
Wikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.
LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.

Input: Who was the band's manager when Black Sabbath released the album that featured the song "Changes"?<end_of_turn>
<start_of_turn>mode

# Start Training

In [15]:
# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_ratio=0.1,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    bf16=False,
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    seed=3407,
    output_dir='./output',
    report_to="none",
    gradient_checkpointing=True,
)

In [16]:
# Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=4096,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, pad_to_multiple_of=8),
    dataset_num_proc=2,
    packing=True,
    args=training_args,
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/90 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/10 [00:00<?, ? examples/s]

In [17]:
# TRAIN ON RESPONSES ONLY
trainer = train_on_responses_only(
    trainer,
    instruction_part="<start_of_turn>user\n",
    response_part="<start_of_turn>model\n",
)

Map (num_proc=6):   0%|          | 0/90 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/10 [00:00<?, ? examples/s]

Input is separated from output

In [18]:
tokenizer.decode(trainer.train_dataset[1]["input_ids"])

'<bos><bos><start_of_turn>user\nFor the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\nWikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.\nLLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.\n\nInput: Which team lost the 2009 Superbowl to the Pittsburgh Steelers?<end_of_turn>\n<start_of_turn>model\nPlan: Search for mo

Only the model response is shown

In [20]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[1]["labels"]]).replace(tokenizer.pad_token, "[MASK]")

'[MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MA

In [21]:
# Training
print("Starting training...")
trainer_stats = trainer.train()
print("Training completed successfully!")

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 90 | Num Epochs = 3 | Total steps = 36
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248 of 4,314,980,720 (0.35% trained)


Step,Training Loss,Validation Loss


Unsloth: Will smartly offload gradients to save VRAM!
Training completed successfully!


In [ ]:
messages = [{
    "role": "system",
    "content": """For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
Wikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.
LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.
 """,
    },
         {
    "role": "user",
    "content": """Who is Pelé?""",
    }
]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation,
    tokenize=False
).removeprefix('<bos>')

outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 300, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1, top_p = 0.95, top_k = 64,
)

output_finetuning = tokenizer.batch_decode(outputs)[0]
print(output_finetuning)

# Testing the model

In [27]:
messages = [{
    "role": "system",
    "content": """For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
Wikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.
LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.
 """,
    },
         {
    "role": "user",
    "content": """Who is Pelé?""",
    }
]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation,
    tokenize=False
).removeprefix('<bos>')

outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 300, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1, top_p = 0.95, top_k = 64,
)

output_finetuning = tokenizer.batch_decode(outputs)[0]
print(output_finetuning)

<bos><start_of_turn>user
For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)
 
Tools can be one of the following:
Wikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.
LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.
 

Who is Pelé?<end_of_turn>
<start_of_turn>model
Plan: Search for more information about Pelé
#E1 = Wikipedia[Pelé]
Plan: Summarize the s

# Saving the model

In [ ]:
if False: # Put True if you want to push to HuggingFace

  from huggingface_hub import notebook_login

  notebook_login()

  model.push_to_hub('username/model-name', use_temp_dir=False)
  tokenizer.push_to_hub('username/model-name', use_temp_dir=False)

In [ ]:
# Save model and artifacts
print("Saving model and artifacts...")

# SALVA IL MODELLO FUSO
print("Merging LoRA weights into base model...")
model.save_pretrained_merged(OUTPUT_DIR, tokenizer)

# Esporta in GGUF (GGUF = formato llama.cpp)
print("Saving model in GGUF format...")
model.save_pretrained_gguf(
    OUTPUT_DIR,              # cartella HF (Hugging Face) con config.json
    tokenizer,
    quantization_method="f16"  # es.: "q4_k_m", "q8_0", "f16"
)